In [9]:
from sqlalchemy.orm import joinedload, selectinload
from sqlmodel import create_engine, Session, SQLModel, select
from dotenv import load_dotenv
import logging
import os
from modelos.post import Post, PostBaseWithUserCommentsTags

In [2]:
logging.basicConfig()
logging.getLogger("sqlalchemy.engine").setLevel(logging.INFO)

In [3]:
load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"))
SQLModel.metadata.create_all(engine)

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("comment")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("user")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("tag")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("posttag")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("post")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:COMMIT


In [16]:
%%time
with Session(engine) as session:
    post = session.get(Post, 1)
    #statement = select(Post).where(Post.id == 1)
    print(post.comments)
    print(post.user)
    print(post.tags)
    print(post.model_dump_json(indent=2))

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT post.id AS post_id, post.title AS post_title, post.content AS post_content, post.created_at AS post_created_at, post.updated_at AS post_updated_at, post.user_id AS post_user_id 
FROM post 
WHERE post.id = ?
INFO:sqlalchemy.engine.Engine:[generated in 0.00375s] (1,)
INFO:sqlalchemy.engine.Engine:SELECT comment.id AS comment_id, comment.content AS comment_content, comment.created_at AS comment_created_at, comment.updated_at AS comment_updated_at, comment.post_id AS comment_post_id, comment.user_id AS comment_user_id 
FROM comment 
WHERE ? = comment.post_id
INFO:sqlalchemy.engine.Engine:[generated in 0.00084s] (1,)
INFO:sqlalchemy.engine.Engine:SELECT user.id AS user_id, user.name AS user_name, user.email AS user_email 
FROM user 
WHERE user.id = ?
INFO:sqlalchemy.engine.Engine:[generated in 0.00072s] (1,)
INFO:sqlalchemy.engine.Engine:SELECT tag.id AS tag_id, tag.name AS tag_name 
FROM tag, posttag 
WHERE

[Comment(id=1, created_at=datetime.datetime(2025, 1, 12, 19, 24, 31, 403369), content='Commentário 1', post_id=1, updated_at=datetime.datetime(2025, 1, 12, 19, 24, 31, 403382), user_id=1), Comment(id=2, created_at=datetime.datetime(2025, 1, 12, 19, 25, 4, 426501), content='Commentário 2', post_id=1, updated_at=datetime.datetime(2025, 1, 12, 19, 25, 4, 426516), user_id=2), Comment(id=3, created_at=datetime.datetime(2025, 1, 12, 19, 25, 10, 911482), content='Commentário 3', post_id=1, updated_at=datetime.datetime(2025, 1, 12, 19, 25, 10, 911493), user_id=3), Comment(id=4, created_at=datetime.datetime(2025, 1, 12, 19, 25, 28, 470366), content='Commentário 4', post_id=1, updated_at=datetime.datetime(2025, 1, 12, 19, 25, 28, 470381), user_id=4), Comment(id=5, created_at=datetime.datetime(2025, 1, 12, 19, 46, 42, 435486), content='Commentário 5', post_id=1, updated_at=datetime.datetime(2025, 1, 12, 19, 46, 42, 435501), user_id=2)]
id=1 email='joão@gmail.com' name='João'
[Tag(id=1, name='gera

In [14]:
%%time
with Session(engine) as session:
    #post = session.get(Post, 1)
    statement = (select(Post).where(Post.id == 1)
                 .options(joinedload(Post.user), joinedload(Post.comments), 
                          joinedload(Post.tags)))
    #statement = select(Post).where(Post.id == 1)
    post = session.exec(statement).first()
    print(post.model_dump_json(indent=2))

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT post.id, post.title, post.content, post.created_at, post.updated_at, post.user_id, user_1.id AS id_1, user_1.name, user_1.email, comment_1.id AS id_2, comment_1.content AS content_1, comment_1.created_at AS created_at_1, comment_1.updated_at AS updated_at_1, comment_1.post_id, comment_1.user_id AS user_id_1, tag_1.id AS id_3, tag_1.name AS name_1 
FROM post LEFT OUTER JOIN user AS user_1 ON user_1.id = post.user_id LEFT OUTER JOIN comment AS comment_1 ON post.id = comment_1.post_id LEFT OUTER JOIN (posttag AS posttag_1 JOIN tag AS tag_1 ON tag_1.id = posttag_1.tag_id) ON post.id = posttag_1.post_id 
WHERE post.id = ?
INFO:sqlalchemy.engine.Engine:[cached since 1.352e+04s ago] (1,)
INFO:sqlalchemy.engine.Engine:ROLLBACK


{
  "id": 1,
  "content": "Conteúdo do 1",
  "updated_at": "2025-01-12T15:07:13.436745",
  "title": "Post 1",
  "created_at": "2025-01-12T15:07:13.436734",
  "user_id": 1
}
CPU times: user 7.25 ms, sys: 6.71 ms, total: 14 ms
Wall time: 12.2 ms


In [15]:
%%time
with Session(engine) as session:
    #post = session.get(Post, 1)
    statement = (select(Post).where(Post.id == 1)
                 .options(selectinload(Post.user), selectinload(Post.comments), 
                          selectinload(Post.tags)))
    #statement = select(Post).where(Post.id == 1)
    post = session.exec(statement).first()
    print(post)
    print(post.model_dump_json(indent=2))

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT post.id, post.title, post.content, post.created_at, post.updated_at, post.user_id 
FROM post 
WHERE post.id = ?
INFO:sqlalchemy.engine.Engine:[cached since 1.324e+04s ago] (1,)
INFO:sqlalchemy.engine.Engine:SELECT post_1.id AS post_1_id, tag.id AS tag_id, tag.name AS tag_name 
FROM post AS post_1 JOIN posttag AS posttag_1 ON post_1.id = posttag_1.post_id JOIN tag ON tag.id = posttag_1.tag_id 
WHERE post_1.id IN (?)
INFO:sqlalchemy.engine.Engine:[cached since 1.324e+04s ago] (1,)
INFO:sqlalchemy.engine.Engine:SELECT user.id AS user_id, user.name AS user_name, user.email AS user_email 
FROM user 
WHERE user.id IN (?)
INFO:sqlalchemy.engine.Engine:[cached since 1.324e+04s ago] (1,)
INFO:sqlalchemy.engine.Engine:SELECT comment.post_id AS comment_post_id, comment.id AS comment_id, comment.content AS comment_content, comment.created_at AS comment_created_at, comment.updated_at AS comment_updated_at, comment.u

id=1 content='Conteúdo do 1' updated_at=datetime.datetime(2025, 1, 12, 15, 7, 13, 436745) title='Post 1' created_at=datetime.datetime(2025, 1, 12, 15, 7, 13, 436734) user_id=1
{
  "id": 1,
  "content": "Conteúdo do 1",
  "updated_at": "2025-01-12T15:07:13.436745",
  "title": "Post 1",
  "created_at": "2025-01-12T15:07:13.436734",
  "user_id": 1
}
CPU times: user 26.3 ms, sys: 9.05 ms, total: 35.3 ms
Wall time: 34.2 ms


In [ ]:
%%time
with Session(engine) as session:
    #post = session.get(Post, 1)
    statement = (select(Post).where(Post.id == 1)
                 .options(selectinload(Post.user), selectinload(Post.comments), 
                          selectinload(Post.tags)))
    #statement = select(Post).where(Post.id == 1)
    post = session.exec(statement).first()
    print(post)
    print(post.model_dump_json(indent=2))

In [4]:
with Session(engine) as session:
    #post = session.get(Post, 1)
    statement = (select(Post).where(Post.id == 1)
                 .options(joinedload(Post.user), joinedload(Post.comments), 
                          joinedload(Post.tags)))
    #statement = select(Post).where(Post.id == 1)
    post = session.exec(statement).first()
    print('---------------------------------------------------------')
    print('post:', post)
    print('post.user:', post.user)
    print('post.comments:', post.comments)
    print('post.tags:', post.tags)
    print('---------------------------------------------------------')
    print(post.model_dump_json(indent=2))
    print('---------------------------------------------------------')
    post2 = PostBaseWithUserCommentsTags(
        **{k: v for k, v in post.__dict__.items() 
           if k in PostBaseWithUserCommentsTags.model_fields.keys()}
    )
    print(post2.model_dump_json(indent=2))

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT post.id, post.title, post.content, post.created_at, post.updated_at, post.user_id, user_1.id AS id_1, user_1.name, user_1.email, comment_1.id AS id_2, comment_1.content AS content_1, comment_1.created_at AS created_at_1, comment_1.updated_at AS updated_at_1, comment_1.post_id, comment_1.user_id AS user_id_1, tag_1.id AS id_3, tag_1.name AS name_1 
FROM post LEFT OUTER JOIN user AS user_1 ON user_1.id = post.user_id LEFT OUTER JOIN comment AS comment_1 ON post.id = comment_1.post_id LEFT OUTER JOIN (posttag AS posttag_1 JOIN tag AS tag_1 ON tag_1.id = posttag_1.tag_id) ON post.id = posttag_1.post_id 
WHERE post.id = ?
INFO:sqlalchemy.engine.Engine:[generated in 0.00200s] (1,)
INFO:sqlalchemy.engine.Engine:SELECT user.id AS user_id, user.name AS user_name, user.email AS user_email 
FROM user 
WHERE user.id = ?
INFO:sqlalchemy.engine.Engine:[generated in 0.00119s] (2,)
INFO:sqlalchemy.engine.Engine:SELECT 

---------------------------------------------------------
post: id=1 content='Conteúdo do 1' updated_at=datetime.datetime(2025, 1, 12, 15, 7, 13, 436745) title='Post 1' created_at=datetime.datetime(2025, 1, 12, 15, 7, 13, 436734) user_id=1
post.user: id=1 email='joão@gmail.com' name='João'
post.comments: [Comment(id=1, created_at=datetime.datetime(2025, 1, 12, 19, 24, 31, 403369), content='Commentário 1', post_id=1, updated_at=datetime.datetime(2025, 1, 12, 19, 24, 31, 403382), user_id=1), Comment(id=2, created_at=datetime.datetime(2025, 1, 12, 19, 25, 4, 426501), content='Commentário 2', post_id=1, updated_at=datetime.datetime(2025, 1, 12, 19, 25, 4, 426516), user_id=2), Comment(id=3, created_at=datetime.datetime(2025, 1, 12, 19, 25, 10, 911482), content='Commentário 3', post_id=1, updated_at=datetime.datetime(2025, 1, 12, 19, 25, 10, 911493), user_id=3), Comment(id=4, created_at=datetime.datetime(2025, 1, 12, 19, 25, 28, 470366), content='Commentário 4', post_id=1, updated_at=datetim